In [7]:
import pandas as pd
import plotly.express as px
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV
from haversine import haversine

In [15]:
df = pd.read_csv('with_nans_in_type_and_with_random_metro.csv')
#df = df.dropna(subset = 'Price')
#df.type = df.type.fillna('monolith')
#df['Number of rooms'].fillna(0, inplace = True)
# df = df[df.Price < 50 * 10**6]
# df = df[df.Area < 500]
df = df.drop(columns = ['Unnamed: 0', 'id', 'date'])

# cors = pd.read_csv('starions_coords.csv', sep = ';') # добавим информацию по станции метро
# cors = cors.drop(columns = ['Unnamed: 3'])
# #подготовим датасеты к присоединению (переведем станции метро в нижний регистр, удалим повторения)
# df['Metro station'] = df['Metro station'].str.lower()
# df['Metro station'] = df['Metro station'].str.replace(' ', '')
# cors['Metro station'] = cors['Metro station'].str.lower()
# cors['Metro station'] = cors['Metro station'].str.replace(' ', '')
# cors = cors.drop_duplicates(subset=['Metro station'])

# districts = pd.read_excel('metro_info.xlsx') # добавим информацию по станции метро
# distances = pd.read_excel('metro_distances.xlsx') # добавим расстояния от станции метро до разных мест
# districts['Metro station'] = districts['Metro station'].str.lower()
# districts['Metro station'] = districts['Metro station'].str.replace(' ', '')
# distances['Metro station'] = distances['Metro station'].str.lower()
# distances['Metro station'] = distances['Metro station'].str.replace(' ', '')
# districts = districts.drop_duplicates(subset=['Metro station'])
# distances = distances.drop_duplicates(subset=['Metro station'])
# df = pd.merge(df, districts, on='Metro station', how='left')
# df = pd.merge(df, distances, on='Metro station', how='left')
# df = df.dropna()


# #присоединим
# df = pd.merge(df, cors, on='Metro station', how='left')
# df = df.dropna()
def calculate_distance(row):
    point_x = (row['lat_x'], row['lon_x'])
    point_y = (row['lat_y'], row['lon_y'])
    return haversine(point_x, point_y, unit='km')

# Применяем функцию к каждой строке DataFrame и создаем новый столбец с расстоянием
#df['distance_km'] = df.apply(calculate_distance, axis=1)

df = df.dropna(subset = 'type') 
df

,Metro station,lon,lat,Number of floors,type,Number of rooms,Area,floor,Price
0,авиамоторная,37.741785,55.736599,45,monolith,0.0,22.0,33,11266065.0
1,авиамоторная,37.735153,55.754122,5,monolith,0.0,21.4,4,6990000.0
2,авиамоторная,37.743276,55.736854,37,monolith,1.0,35.0,2,13215071.0
3,авиамоторная,37.700595,55.749435,28,monolith,1.0,34.0,6,18990000.0
4,авиамоторная,37.696511,55.753244,11,monolith,1.0,38.9,3,12436330.0
...,...,...,...,...,...,...,...,...,...
74602,парккультуры,37.586338,55.740517,17,panel,2.0,45.7,10,26000000.0
74603,киевская,37.576313,55.741029,9,brick,2.0,51.0,2,31000000.0
74604,спортивная,37.576879,55.718633,16,brick,2.0,57.0,3,39000000.0
74606,киевская,37.574489,55.739843,10,brick,3.0,61.4,2,40000000.0


In [16]:
X = df.drop(columns=['Price'])  # Признаки 
y = df['Price']  # Таргет
#разбиение на тестовую и обучающую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
#Обучим модель, используя CatBoost
model = CatBoostRegressor(iterations=3000,  # количество деревьев
                          learning_rate=0.05,  # скорость обучения
                          depth=10,  # глубина деревьев
                          random_state=42,  # для воспроизводимости результатов
                          verbose=250)  # подробность вывода информации о процессе обучения
# Обучение
model.fit(X_train, y_train, cat_features=[0, 4,])  # Указываем индексы категориальных признаков

# Прогнозa
y_pred = model.predict(X_test)

# Оценка производительности модели
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
print('\nРЕЗУЛЬТАТЫ Gradient Boosting:', 
    '\nMSE: ', mse,
     '\nRMSE: ', rmse, 
     '\nMAE: ', mae, 
     '\nMAPE: ', mape,
     '\n\nRMSE in millions: ', (rmse/1000000).round(3), 'millions')

0:	learn: 9470095.1775665	total: 264ms	remaining: 13m 12s


In [152]:
y_pred = model.predict(X_train)
mse = mean_squared_error(y_train, y_pred)
rmse = np.sqrt(mean_squared_error(y_train, y_pred))
mae = mean_absolute_error(y_train, y_pred)
mape = np.mean(np.abs((y_train - y_pred) / y_train)) * 100
print('\nРЕЗУЛЬТАТЫ Gradient Boosting на трейне:', 
    '\nMSE: ', mse,
     '\nRMSE: ', rmse, 
     '\nMAE: ', mae, 
     '\nMAPE: ', mape,
     '\n\nRMSE in millions: ', (rmse/1000000).round(3), 'millions')


РЕЗУЛЬТАТЫ Gradient Boosting на трейне: 
MSE:  1677778025406.6895 
RMSE:  1295290.71076986 
MAE:  813095.9213516416 
MAPE:  6.623794742095042 

RMSE in millions:  1.295 millions


In [146]:
feature_importances = model.feature_importances_
feature_importance_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': feature_importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
px.histogram(feature_importance_df, x='Feature', y='Importance', template='simple_white',
             title='Важность фичей в градиентном бустинге в нашем случае', color_discrete_sequence=['#3c43a5'], 
             labels = {'count':'количество'})

In [95]:
request = ['Авиамоторная', 37.741785, 55.736599, 45, 'monolith', 2, 40, 33]

columns = ['Metro station', 'lon', 'lat', 'Number of floors',
           'type', 'Number of rooms', 'Area', 'floor']

request_df = pd.DataFrame([request], columns=columns)
request_df['Metro station'] = request_df['Metro station'].str.lower()
request_df['Metro station'] = request_df['Metro station'].str.replace(' ', '')
cors['Metro station'] = cors['Metro station'].str.lower()
cors['Metro station'] = cors['Metro station'].str.replace(' ', '')
cors = cors.drop_duplicates(subset=['Metro station'])

request_df = pd.merge(request_df, cors, on='Metro station', how='left')
request_df['distance_km'] = request_df.apply(calculate_distance, axis=1)
#присоединим
predict = model.predict(request_df)[0]

def interval(predict):
    interval_list = []
    interval_list.append(np.round((predict * 0.99)/1000000, 1))
    interval_list.append(np.round((predict * 1.01)/1000000, 1))
    return interval_list

interval(predict)

# ИНТЕРВАЛ ЦЕН В МЛН РУБЛЕЙ, ВЫВОД МОДЕЛИ

[15.7, 16.1]

In [93]:
#model.save_model('model.bin')
model = CatBoostRegressor()
model.load_model('model.bin')

In [128]:
fig = px.scatter(x = y_pred, y = y_train, template = 'simple_white')

fig.show()

In [123]:
y

0        11266065.0
1         6990000.0
2        13215071.0
3        18990000.0
4        12436330.0
            ...    
25829     9798667.0
25830    19115233.0
25831    13408830.0
25832    13334644.0
25833    15520514.0
Name: Price, Length: 25834, dtype: float64

In [124]:
y_pred

array([19743756.60827236, 25416942.89674227, 12349501.63605828, ...,
       34706863.30667844,  9255537.41866114, 40437952.57319592])

In [1]:
param_grid = {
    'iterations': [5000, 8000, 11000, 15000],  
    'learning_rate': [0.05, 0.1, 0.2, 0.005],
    'depth': [9, 11, 15, 6]
} #такие значения на втором переборе

model = CatBoostRegressor(random_state=42, verbose=100)

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=2, scoring='neg_mean_squared_error', verbose=2)

grid_search.fit(X_train, y_train, cat_features= cat_features=[0, 4,])

best_params = grid_search.best_params_
print("Лучшие гиперпараметры:", best_params)

best_model = grid_search.best_estimator_

y_pred = best_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
print('\nРЕЗУЛЬТАТЫ Gradient Boosting с CatBoost (лучшая модель):', 
    '\nMSE: ', mse,
    '\nRMSE: ', rmse, 
    '\nMAE: ', mae, 
    '\nMAPE: ', mape,
    '\n\nRMSE in millions: ', (rmse/1000000).round(3), 'millions')

SyntaxError: invalid syntax (3368670917.py, line 2)